### This notebook is for sub-volume extraction and radiomics calculation
### Author: Zhenwei Shi, Tianchen Luo, Xiaomei Huang,Zhihe Zhao


In [ ]:
import os
import SimpleITK as sitk
import pandas as pd
import glob

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, color
import skimage.morphology as morphology

from skimage.segmentation import mark_boundaries as mark_boundaries
from skimage.segmentation import slic as slic

import shutil


In [ ]:
def SV_split(volume, mask, out_dir):

    mask_array = sitk.GetArrayFromImage(sitk.ReadImage(mask))
    _, num = measure.label(mask_array, connectivity=3, return_num=True) # meature label connected regions

    for i in range(1, num + 1):
        section_volume = sitk.GetArrayFromImage(sitk.ReadImage(volume))
        section_mask = sitk.GetArrayFromImage(sitk.ReadImage(mask))

        section_mask[section_mask != i] = 0 # split sv mask
        section_mask[section_mask == i] = 1 # split sv mask

        # ----- for SV mask split ------
        print(mask.split('\\')[-1].split('.')[-3])
        svsplit_path = os.path.join(out_dir,'SVmask',mask.split('\\')[-1].split('.')[-3].split('_')[-2]) # sv path per patient
        
        if not os.path.exists(svsplit_path):
            os.makedirs(svsplit_path)        
        
        section_mask_img = sitk.GetImageFromArray(section_mask)
        
        section_mask_img.SetOrigin(sitk.ReadImage(mask).GetOrigin())
        section_mask_img.SetSpacing(sitk.ReadImage(mask).GetSpacing())
        section_mask_img.SetDirection(sitk.ReadImage(mask).GetDirection())
        print(sitk.ReadImage(mask).GetOrigin())
        sitk.WriteImage(section_mask_img, # save sv mask separately
                os.path.join(svsplit_path, mask.split('\\')[-1].split('.')[-3]) + str(i)+'.nii.gz')

        # ----- for SV volume split ------
        # print(volume.split('/')[-1].split('.')[-3])
    
        # # section_volume[section_mask == 0] = 0
        # sitk.WriteImage(sitk.GetImageFromArray(section_volume), # save sv mask separately
        #         os.path.join(svsplit_path, 'SVvolume', volume.split('/')[-1].split('.')[-3])+'_SVvolume_'+ str(i)+'.nii.gz')

def check_dataset(list_volume, list_mask):
    if len(list_volume) != len(list_mask):
        raise ValueError('There exists a mismatch between two datasets.')

def mac_os_listdir(path):
    files = os.listdir(path)
    if '.DS_Store' in files:
        files.remove('.DS_Store')
        files.sort()
    return files

def get_file_num(path):
    files_num = []
    files = os.listdir(path)
    for file in files:
        if file !='.DS_Store':
            # print(file)
            num = ""
            for c in file:
                if c.isdigit():
                    num = num + c
            # print("Extracted numbers from the list : " + num)
            files_num.append(num)
    return files_num

def supervoxel(volume_list,mask_list,n_segment,supervoxel_path,center):
    print('Start super voxel processing ...')
    if not os.path.exists(supervoxel_path):
        os.makedirs(supervoxel_path)
    
    if not os.path.exists(os.path.join(supervoxel_path,center)):
        os.makedirs(os.path.join(supervoxel_path,center))

    for volume_path, mask_path in zip(volume_list, mask_list):
        
        img = sitk.GetArrayFromImage(sitk.ReadImage(volume_path))
        mask = sitk.GetArrayFromImage(sitk.ReadImage(mask_path))
        img_normalized = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        tmp_mask = morphology.opening(mask, morphology.ball(1))

        try:
            slic_mask = slic(img_normalized, n_segment, compactness=10, mask=tmp_mask, start_label=1,
                             channel_axis=None)
        except:
            print('something wrong, SV computation failed !!!')

        mask_itk = sitk.GetImageFromArray(np.uint16(slic_mask))
        # print(sitk.ReadImage(mask_path).GetOrigin())
        mask_itk.SetOrigin(sitk.ReadImage(mask_path).GetOrigin())
        mask_itk.SetSpacing(sitk.ReadImage(mask_path).GetSpacing())
        mask_itk.SetDirection(sitk.ReadImage(mask_path).GetDirection())
        # save slic_mask
        output_name = os.path.join(supervoxel_path,center,center+'_' + mask_path.split('\\')[-1].split('.')[0] + '_SVmask.nii.gz')
        sitk.WriteImage(mask_itk, output_name)
        print('{} is Done'.format(output_name.split('\\')[-1]))

#TODO
# def resampe():
# def normalization():

#### Set User Parameters

In [ ]:
# User parameters - path

# data_dir = r'/media/zhenwei/3b5e8580-3829-4c0a-b2fa-ce0cd26a7d93/Data/Breast_MRI/SY'  # specify where your data store
data_dir = r'D:\03_AIDataSet\00_Radiomics\Dataset\QMITH'
code_dir = r'D:\\00_tianyi_cloud\\00_github\\16_radiomics\QMITH\Python_script' # as a working directory

# User parameters - sub-vlume
n_segment = 100
center = 'hn'
# center = 'hn'

# User parameters - radiomics

# casetable_dir = r'../output/CaseTable'
# rf_dir = r'../output/RF'
casetable_dir= os.path.join(data_dir,'output','CaseTable')
rf_dir = os.path.join(data_dir,'output','RF')
# --------------------------------------------------------
tmp_volume = os.path.join(data_dir,'tmp_dir','VolumeDir')
tmp_mask = os.path.join(data_dir,'tmp_dir','MaskDir')
out_dir = os.path.join(data_dir,'output')
SVS_dir = os.path.join(out_dir,'SV','SVSplit')
SVSmask_dir = os.path.join(out_dir,'SV','SVSplit','SVmask')
SVSvolume_dir = os.path.join(out_dir,'SV','SVSplit','SVvolume')
SVwhole_dir = os.path.join(out_dir,'SV','SVWhole')

# --------------------------------------------------------
## remove files from temparal directory
if os.path.isdir(tmp_volume):
    shutil.rmtree(tmp_volume)

if os.path.isdir(tmp_mask):
    shutil.rmtree(tmp_mask)
# --------------------------------------------------------
# check if dirs exist
dir_list = [out_dir,tmp_volume,tmp_mask,SVS_dir,SVSmask_dir,SVSvolume_dir,SVwhole_dir,casetable_dir,rf_dir]
for dir in dir_list:
    if not os.path.exists(dir):
        os.makedirs(dir)
        

In [ ]:
# for users 
# specify image nii dir and mask nii dir

volume_list= glob.glob(os.path.join(data_dir,'image','*'))
mask_list = glob.glob(os.path.join(data_dir,'mask','*'))

volume_list.sort()
mask_list.sort()

# check if they are in the same length
#TODO check id and pre/post
check_dataset(volume_list, mask_list)

#### Implementation of sub-volume calculation

In [ ]:
# copy files to fixed and temparal dirs for easy process 
for volume, mask in zip(volume_list,mask_list):
    shutil.copy2(volume,os.path.join(data_dir,'tmp_dir','VolumeDir'))
    shutil.copy2(mask,os.path.join(data_dir,'tmp_dir','MaskDir'))

volume_list_new = glob.glob(os.path.join(data_dir,'tmp_dir','VolumeDir','*'))
mask_list_new = glob.glob(os.path.join(data_dir,'tmp_dir','MaskDir','*'))
volume_list_new.sort()
mask_list_new.sort()

# check if they are in the same length
check_dataset(volume_list_new, mask_list_new)
print('Dataset checked!')

# supervoxel calculation by slic, 
supervoxel(volume_list_new, mask_list_new, n_segment,SVwhole_dir,center)
print('Supervoxel Done')

sv_list = mac_os_listdir(os.path.join(SVwhole_dir,center))
sv_list.sort()
sv_list = [os.path.join(SVwhole_dir,center, file) for file in sv_list]
check_dataset(volume_list_new, sv_list)

# supervoxel split for further processing
print('Start SV split processing ...')
for volume_i, mask_j in zip(volume_list_new, sv_list):
    SV_split(volume_i, mask_j, SVS_dir)
print('SV split done')

#### Generate case table for radiomics

In [ ]:
# for radiomics - prepare casetable
image_files = volume_list_new
mask_files = os.listdir(SVSmask_dir)
image_files.sort()
mask_files.sort()

check_dataset(image_files, mask_files)

for i in range(0,len(image_files)):
    img_id = image_files[i].split('\\')[-1].split('_')[-1].split('.')[-2]
    # print(image_files[i].split('\\')[-1].split('_')[-1].split('.')[-2])
    SV_files = mac_os_listdir(os.path.join(SVSmask_dir,mask_files[i]))
    SV_files.sort()
    CaseTable = pd.DataFrame()
    for j in range(0,len(SV_files)):
        ptid = pd.Series({'ID':img_id+'_'+SV_files[j].split('.')[-3]})
        # print(SV_files[j].split('.')[-3])
        
        Image_path = pd.Series({'Image':image_files[i]})
       
        Mask_path = pd.Series({'Mask':os.path.join(SVSmask_dir,mask_files[i],SV_files[j])})

        # case_id_img = ptid.append(Image_path)
        case_id_img =  pd.concat([ptid, Image_path], axis=0, ignore_index=True)

        # case_id_img_mask = case_id_img.append(Mask_path)
        case_id_img_mask=pd.concat([case_id_img, Mask_path], axis=0, ignore_index=True)
        
        # print(case_id_img_mask)
        CaseTable = pd.concat([CaseTable,case_id_img_mask],axis = 1)
        
        # print(CaseTable)    # generate case table for radiomics calculation, including ID, image and mask path
    # CaseTable=CaseTable.iloc[1:] 
    # print(CaseTableT)
    # CaseTableT =  CaseTableT
    print(CaseTable)
    CaseTableT=CaseTable.T
    CaseTableT.columns = ['ID', 'Image', 'Mask']
    CaseTableT.to_csv(os.path.join(casetable_dir,img_id +'_CaseTable.csv'),index=False)

#### Implementation of radiomics calculation

In [ ]:
# for radiomics calculation
# recommand to run it on command line
%run ../Pre_processing/batchProcessingWithPandas_split.py